In [2]:
import polars as pl
import pandas as pd
import polars.datatypes as T
import numpy as np

from datetime import datetime, time
import sys
sys.path.insert(0, 'C:\\Users\\rockh\\Repositories\\Tsuro\\')

from tsuro.datasets import load_cba_trades
from tsuro.preprocessing import cast_strings_to_datetime

C:\Users\rockh\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
import random
test = np.random.randint(60, size = 30)
test

array([21, 56, 20,  1, 38, 28, 50, 41, 12, 39, 52,  9, 31, 37, 53,  1, 30,
       40,  6, 38, 42, 19, 22, 18, 33, 59, 29, 22, 23, 17])

In [12]:
def create_pseudo_bars(df):
    
    df = df.with_columns(
        pl.lit(
            np.random.randint(low = 1, high = 60, size = df.height)
        ).alias("add_minutes"),
        pl.lit(
            np.random.randint(6, size = df.height)
        ).alias("add_hours"),
    )
    
    df = cast_strings_to_datetime(
        df, 
        columns = ["time"]
    )
    
    df = df.with_columns(
        (
            pl.col("time") + 
            pl.duration(
                hours = pl.col("add_hours"), 
                minutes = pl.col("add_minutes")
            )
        ).alias("datetime_end")
    )
    
    df = df.rename(
        mapping = {"time": "datetime_start"}
    )
    
    return df

from tsuro.utils import create_dataframe_index
df = load_cba_trades()
df_test = create_pseudo_bars(df).select("datetime_start", "datetime_end")
df_test = create_dataframe_index(df_test)

df_test[0:3]

datetime_start,datetime_end,index
datetime[μs],datetime[μs],u32
2021-06-10 17:00:00,2021-06-10 17:45:00,0
2021-06-10 16:54:00,2021-06-10 17:13:00,1
2021-06-10 16:47:00,2021-06-10 17:21:00,2


In [2]:
import time
from tsuro.sampling import create_overlap_matrix

time0 = time.time()
overlap_matrix = create_overlap_matrix(
        df_test,
        time_start_col = "datetime_start",
        time_end_col = "datetime_end",
        index_col = "index",
        remove_no_overlaps = True
    )
time1 = time.time()
completion_time = time1-time0
print(f"Completion Time: {completion_time}")


NameError: name 'df_test' is not defined

In [3]:
#overlap_matrix.write_csv("overlap_matrix_test.csv")
overlap_matrix = pl.read_csv("overlap_matrix_test.csv")
overlap_matrix

index,time_start,time_end,bar_0_overlap,bar_1_overlap,bar_2_overlap,bar_3_overlap,bar_4_overlap,bar_5_overlap,bar_6_overlap,bar_7_overlap,bar_8_overlap,bar_9_overlap,bar_10_overlap,bar_11_overlap,bar_12_overlap,bar_13_overlap,bar_14_overlap,bar_15_overlap,bar_16_overlap,bar_17_overlap,bar_18_overlap,bar_19_overlap,bar_20_overlap,bar_21_overlap,bar_22_overlap,bar_23_overlap,bar_24_overlap,bar_25_overlap,bar_26_overlap,bar_27_overlap,bar_28_overlap,bar_29_overlap,bar_30_overlap,bar_31_overlap,bar_32_overlap,bar_33_overlap,…,bar_66540_overlap,bar_66541_overlap,bar_66542_overlap,bar_66543_overlap,bar_66544_overlap,bar_66545_overlap,bar_66546_overlap,bar_66547_overlap,bar_66548_overlap,bar_66549_overlap,bar_66550_overlap,bar_66551_overlap,bar_66552_overlap,bar_66553_overlap,bar_66554_overlap,bar_66555_overlap,bar_66556_overlap,bar_66557_overlap,bar_66558_overlap,bar_66559_overlap,bar_66560_overlap,bar_66561_overlap,bar_66562_overlap,bar_66563_overlap,bar_66564_overlap,bar_66565_overlap,bar_66566_overlap,bar_66567_overlap,bar_66568_overlap,bar_66569_overlap,bar_66570_overlap,bar_66571_overlap,bar_66572_overlap,bar_66573_overlap,bar_66574_overlap,bar_66575_overlap,bar_66576_overlap
i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,…,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
0,"""2021-06-10T07:05:00.000000""","""2021-06-10T07:17:00.000000""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1,"""2021-06-10T07:17:00.000000""","""2021-06-10T07:19:00.000000""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1
2,"""2021-06-10T07:19:00.000000""","""2021-06-10T07:26:00.000000""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1
3,"""2021-06-10T07:26:00.000000""","""2021-06-10T07:31:00.000000""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1
4,"""2021-06-10T07:31:00.000000""","""2021-06-10T07:46:00.000000""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
739,"""2021-06-10T21:59:00.000000""","""2021-06-10T22:00:00.000000""",0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
740,"""2021-06-10T22:00:00.000000""","""2021-06-10T22:04:00.000000""",0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
741,"""2021-06-10T22:04:00.000000""","""2021-06-10T22:06:00.000000""",0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [4]:
overlap_matrix = pl.read_csv("overlap_matrix_test.csv")

overlap_matrix_bars = overlap_matrix.with_columns(
    pl.when(
        pl.col("time_start") == pl.col("time_end"),
    ).then(1).otherwise(0).alias("zero_time_interval_flag")
).filter(
    pl.col("zero_time_interval_flag") != 1
).drop(
    "index",
    "time_start",
    "time_end",
    "zero_time_interval_flag"
)[:, 0:1000]

overlap_matrix_vert_sum = overlap_matrix_bars.sum()

overlap_matrix_bars

bar_0_overlap,bar_1_overlap,bar_2_overlap,bar_3_overlap,bar_4_overlap,bar_5_overlap,bar_6_overlap,bar_7_overlap,bar_8_overlap,bar_9_overlap,bar_10_overlap,bar_11_overlap,bar_12_overlap,bar_13_overlap,bar_14_overlap,bar_15_overlap,bar_16_overlap,bar_17_overlap,bar_18_overlap,bar_19_overlap,bar_20_overlap,bar_21_overlap,bar_22_overlap,bar_23_overlap,bar_24_overlap,bar_25_overlap,bar_26_overlap,bar_27_overlap,bar_28_overlap,bar_29_overlap,bar_30_overlap,bar_31_overlap,bar_32_overlap,bar_33_overlap,bar_34_overlap,bar_35_overlap,bar_36_overlap,…,bar_963_overlap,bar_964_overlap,bar_965_overlap,bar_966_overlap,bar_967_overlap,bar_968_overlap,bar_969_overlap,bar_970_overlap,bar_971_overlap,bar_972_overlap,bar_973_overlap,bar_974_overlap,bar_975_overlap,bar_976_overlap,bar_977_overlap,bar_978_overlap,bar_979_overlap,bar_980_overlap,bar_981_overlap,bar_982_overlap,bar_983_overlap,bar_984_overlap,bar_985_overlap,bar_986_overlap,bar_987_overlap,bar_988_overlap,bar_989_overlap,bar_990_overlap,bar_991_overlap,bar_992_overlap,bar_993_overlap,bar_994_overlap,bar_995_overlap,bar_996_overlap,bar_997_overlap,bar_998_overlap,bar_999_overlap
i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,…,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [5]:
overlap_matrix = pl.read_csv("overlap_matrix_test.csv")

overlap_matrix_bars = overlap_matrix.with_columns(
    pl.when(
        pl.col("time_start") == pl.col("time_end"),
    ).then(1).otherwise(0).alias("zero_time_interval_flag")
).filter(
    pl.col("zero_time_interval_flag") != 1
).drop(
    "index",
    "time_start",
    "time_end",
    "zero_time_interval_flag"
)[:, 0:1000]

overlap_matrix_vert_sum = overlap_matrix_bars.sum()

overlap_matrix_bars_np = overlap_matrix_bars.to_numpy()
overlap_matrix_vert_sum_np = overlap_matrix_vert_sum.to_numpy()
overlap_matrix_bars_np

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [7]:
for i in range(overlap_matrix_bars_np.shape[1]):
    overlap_matrix_bars_np[0,i] = i

array([[1, 3, 5],
       [0, 0, 0],
       [0, 0, 0],
       ...,
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1]], dtype=int64)

In [14]:
from sklearn.utils.random import sample_without_replacement

sample_indices = sample_without_replacement(
    1000, 10
)

overlap_matrix_bars_np[:,sample_indices]

array([[612, 517, 823, ..., 722, 199, 289],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       ...,
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0]], dtype=int64)

In [19]:
verbose = 2
n_estimators = 30
total_n_estimators = 100
for i in range(n_estimators):
    if verbose > 1:
        print(
            f"Building estimator {i+1} of {n_estimators} for this parallel run (total {total_n_estimators})..."
        )

Building estimator 1 of 30 for this parallel run (total 100)...
Building estimator 2 of 30 for this parallel run (total 100)...
Building estimator 3 of 30 for this parallel run (total 100)...
Building estimator 4 of 30 for this parallel run (total 100)...
Building estimator 5 of 30 for this parallel run (total 100)...
Building estimator 6 of 30 for this parallel run (total 100)...
Building estimator 7 of 30 for this parallel run (total 100)...
Building estimator 8 of 30 for this parallel run (total 100)...
Building estimator 9 of 30 for this parallel run (total 100)...
Building estimator 10 of 30 for this parallel run (total 100)...
Building estimator 11 of 30 for this parallel run (total 100)...
Building estimator 12 of 30 for this parallel run (total 100)...
Building estimator 13 of 30 for this parallel run (total 100)...
Building estimator 14 of 30 for this parallel run (total 100)...
Building estimator 15 of 30 for this parallel run (total 100)...
Building estimator 16 of 30 for th

In [14]:
rs = np.random.RandomState(seed = 5)

result = rs.choice(range(3), p = np.array([0,1/2,1/2]))

result

1

In [17]:
import time
from tsuro.sampling.bootstrap_sampling import create_sequential_bootstrap_sample_indices
random_state = np.random.RandomState()
index_pool = np.arange(overlap_matrix_bars.width)
time0 = time.time()
sample_indices = create_sequential_bootstrap_sample_indices(
    overlap_matrix = overlap_matrix_bars_np,
    overlap_matrix_column_sums = overlap_matrix_vert_sum_np,
    index_pool = index_pool,
    random_state = random_state
)
time1 = time.time()
completion_time = time1-time0
print(f"Completion Time: {completion_time}")
sample_indices

Completion Time: 5.726243734359741


array([ 84, 491, 789, 445, 263, 759, 635, 717, 238, 812, 227, 384, 737,
       120, 481, 553, 823,  39, 667, 754, 208, 270, 759, 831, 607, 789,
         8, 294, 529,  12, 907, 219, 151, 581, 101, 766, 904, 970, 464,
       243, 545, 136,  22, 592, 502, 575, 853, 862, 772, 871, 123, 369,
       835, 135, 447, 447, 769, 743, 812, 203, 451,  20, 360, 714, 512,
       664, 781, 849, 764, 657, 351, 602, 870, 189,  31, 850, 161, 882,
       392, 962, 650, 555, 559, 677,  51,  45, 750, 947, 673, 900, 521,
       286, 190, 936, 209, 640, 449, 327, 943, 936, 604, 824,  27, 342,
       190, 563, 713, 408, 348, 436, 518, 660,  79, 206,  46, 867, 110,
        92,   0, 146, 379, 897,  79, 550, 278, 995, 863, 822,  64, 606,
       382, 838, 476, 773, 114, 592, 963, 791,  47, 429, 507, 615, 990,
        22, 931, 128, 604, 867, 532,  13, 798,  59, 463, 745,  20,  95,
       597, 859, 580, 121, 567, 845,  79, 617, 946, 356, 869, 361, 625,
        82, 762, 972, 235, 294, 402,  24, 603, 290, 295, 843, 48

In [4]:
import time
from tsuro.sampling import create_overlap_matrix

complexity = {}

sample_sizes = [100, 500, 1000, 5000, 10000, 25000, 50000]

for sample_size in sample_sizes:
    print(f"Sample Size: {sample_size}")
    df_sample = df_test[0:sample_size]
    
    time0 = time.time()
    overlap_mat = create_overlap_matrix(
        df_sample,
        time_start_col = "datetime_start",
        time_end_col = "datetime_end",
        index_col = "index",
        remove_no_overlaps = True
    )
    time1 = time.time()
    
    completion_time = time1-time0
    print(f"Completion Time: {completion_time}")
    complexity[str(sample_size)] = completion_time

complexity

Sample Size: 100
Completion Time: 0.06164908409118652
Sample Size: 500
Completion Time: 0.2740144729614258
Sample Size: 1000
Completion Time: 0.5509986877441406
Sample Size: 5000
Completion Time: 2.934000015258789
Sample Size: 10000
Completion Time: 6.29799747467041
Sample Size: 25000
Completion Time: 17.437002420425415
Sample Size: 50000
Completion Time: 72.2759518623352


{'100': 0.06164908409118652,
 '500': 0.2740144729614258,
 '1000': 0.5509986877441406,
 '5000': 2.934000015258789,
 '10000': 6.29799747467041,
 '25000': 17.437002420425415,
 '50000': 72.2759518623352}

PSEUDO CODE
Function receives two arguments:
1) sample_bars [[y_1, tau_start_1, tau_end_1],[y_2, tau_start_2, tau_end_2] ..., [y_I, tau_start_I, tau_end_I]].
2) time_index [tau_0,tau_1,...,tau_t].

Easier to work with index label itself. We have no need for timestamps. At the end of the day we just want to create bootstrap samples for the sample_bars. Therefore, time_index is optional argument. Pass sample_bars through alone with tau_start, tau_end replaced with time index labels ([y_1, 1, 2] means that y_1 bar covers the period [tau_1,tau_2]). We will build at most a t x I indicator matrix. We only care about the period [tau_start_1, tau_end_I], so we can further reduce the complexity.

In [7]:
from tsuro.sampling import create_overlap_matrix
bars_df = pl.DataFrame({
        "bar_index": [0,1,2],
        "volume": [10, 20, 50],
        "time_start": ["10:00", "10:02", "10:03"],
        "time_end": ["10:03", "10:10", "10:15"]
        }
)

index = pl.Series(values = ["9:55", "10:00", "10:02", "10:03", "10:04", "10:05", "10:08", "10:10", "10:12", "10:15"])
overlap_matrix = create_overlap_matrix(
    bars_df,
    time_start_col = "time_start",
    time_end_col = "time_end",
    time_index = index,
    index_col = "bar_index",
    remove_no_overlaps = True
)
overlap_matrix.width

6

In [8]:
from tsuro.data_structures import StandardBars

# Create Trades DataFrame
trades_df = load_cba_trades()

trades_df = cast_strings_to_datetime(trades_df, columns = ["time"])

time_index = trades_df.select("time").unique()
type(time_index)

polars.dataframe.frame.DataFrame

In [9]:
bars = StandardBars(datetime_col = "time", price_col = "price", volume_col = "volume")

trade_bars = bars.create_volume_bars(
    trades_df,
    order_by = "time"
).with_columns(
    pl.when(
        pl.col("datetime_start") == pl.col("datetime_end"),
    ).then(1).otherwise(0).alias("zero_time_interval_flag")
).filter(
    pl.col("zero_time_interval_flag") != 1
)
trade_bars

bar_index,open_price,high_price,low_price,close_price,datetime_start,datetime_end,zero_time_interval_flag
i32,f64,f64,f64,f64,datetime[μs],datetime[μs],i32
30,99.89,99.98,99.77,99.915,2021-06-10 12:47:00,2021-06-10 12:58:00,0
21,99.845,99.94,99.62,99.7,2021-06-10 11:10:00,2021-06-10 11:14:00,0
12,100.8,101.08,100.72,100.945,2021-06-10 10:29:00,2021-06-10 10:33:00,0
15,100.905,100.96,100.35,100.35,2021-06-10 10:45:00,2021-06-10 10:49:00,0
9,101.6,101.6,101.01,101.21,2021-06-10 10:18:00,2021-06-10 10:22:00,0
…,…,…,…,…,…,…,…
47,99.81,100.08,99.76,100.08,2021-06-10 15:55:00,2021-06-10 16:10:00,0
38,99.91,99.97,99.73,99.765,2021-06-10 14:29:00,2021-06-10 14:39:00,0
41,99.92,100.02,99.84,99.9,2021-06-10 15:06:00,2021-06-10 15:16:00,0


In [10]:
import time

time0 = time.time()
trades_overlap_matrix = create_overlap_matrix(
    trade_bars,
    time_start_col = "datetime_start",
    time_end_col = "datetime_end",
    time_index = time_index,
    index_col = "bar_index",
    remove_no_overlaps = True
)

time1 = time.time()
print(f"Time: {time1-time0}")
trades_overlap_matrix

Time: 0.04100155830383301


index,time_start,time_end,bar_2_overlap,bar_4_overlap,bar_5_overlap,bar_6_overlap,bar_7_overlap,bar_8_overlap,bar_9_overlap,bar_10_overlap,bar_11_overlap,bar_12_overlap,bar_13_overlap,bar_14_overlap,bar_15_overlap,bar_16_overlap,bar_17_overlap,bar_18_overlap,bar_19_overlap,bar_20_overlap,bar_21_overlap,bar_22_overlap,bar_23_overlap,bar_24_overlap,bar_25_overlap,bar_26_overlap,bar_27_overlap,bar_28_overlap,bar_29_overlap,bar_30_overlap,bar_31_overlap,bar_32_overlap,bar_33_overlap,bar_34_overlap,bar_35_overlap,bar_36_overlap,bar_37_overlap,bar_38_overlap,bar_39_overlap,bar_40_overlap,bar_41_overlap,bar_42_overlap,bar_43_overlap,bar_44_overlap,bar_45_overlap,bar_46_overlap,bar_47_overlap,bar_61_overlap,bar_65_overlap,bar_66_overlap,bar_67_overlap
u32,datetime[μs],datetime[μs],i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32
0,2021-06-10 07:05:00,2021-06-10 10:02:00,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2021-06-10 10:02:00,2021-06-10 10:03:00,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2021-06-10 10:03:00,2021-06-10 10:04:00,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2021-06-10 10:04:00,2021-06-10 10:05:00,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2021-06-10 10:05:00,2021-06-10 10:06:00,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
372,2021-06-10 16:37:00,2021-06-10 16:43:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
373,2021-06-10 16:43:00,2021-06-10 16:45:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
374,2021-06-10 16:45:00,2021-06-10 16:46:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [11]:
from tsuro.utils import compute_column_sums
from tsuro.sampling.bootstrap_sampling import create_sequential_bootstrap_sample_indices

trades_overlap_matrix_bars = trades_overlap_matrix.drop("index", "time_start", "time_end")
trades_overlap_matrix_bars_sums = compute_column_sums(trades_overlap_matrix_bars)
trades_overlap_matrix_bars_sums

bar_2_overlap,bar_4_overlap,bar_5_overlap,bar_6_overlap,bar_7_overlap,bar_8_overlap,bar_9_overlap,bar_10_overlap,bar_11_overlap,bar_12_overlap,bar_13_overlap,bar_14_overlap,bar_15_overlap,bar_16_overlap,bar_17_overlap,bar_18_overlap,bar_19_overlap,bar_20_overlap,bar_21_overlap,bar_22_overlap,bar_23_overlap,bar_24_overlap,bar_25_overlap,bar_26_overlap,bar_27_overlap,bar_28_overlap,bar_29_overlap,bar_30_overlap,bar_31_overlap,bar_32_overlap,bar_33_overlap,bar_34_overlap,bar_35_overlap,bar_36_overlap,bar_37_overlap,bar_38_overlap,bar_39_overlap,bar_40_overlap,bar_41_overlap,bar_42_overlap,bar_43_overlap,bar_44_overlap,bar_45_overlap,bar_46_overlap,bar_47_overlap,bar_61_overlap,bar_65_overlap,bar_66_overlap,bar_67_overlap
i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32
1,3,2,5,3,3,4,3,4,4,5,7,4,5,6,3,4,3,4,7,11,12,11,11,10,16,15,11,14,15,9,12,12,10,19,10,16,11,10,11,9,7,7,5,5,1,4,5,2


In [15]:
test = np.zeros([3,2])
test

array([[0., 0.],
       [0., 0.],
       [0., 0.]])

In [12]:
import numpy as np

index_pool = np.arange(trades_overlap_matrix_bars.width)

time0 = time.time()

for i in range(100):
    sample_indices = create_sequential_bootstrap_sample_indices(
        trades_overlap_matrix_bars,
        trades_overlap_matrix_bars_sums,
        index_pool = index_pool
    )
time1 = time.time()

print(f"Time: {time1-time0}")
sample_indices

Time: 14.155001163482666


array([18, 35, 11, 25, 21,  6, 26,  9,  6, 19, 24, 14,  3, 20,  7,  4, 12,
       37, 34, 33, 30, 45, 41, 32,  8, 39,  2, 43, 38,  7, 11, 31, 44, 38,
       28, 46, 36, 16,  6,  6,  4, 23, 40, 37, 19,  0, 47,  8, 44])

In [23]:
def create_bar_sizes(
        overlap_matrix,
        bar_col_start_idx: int = 0
):
    """
    Compute sizes for each bar
    """
    bar_cols = overlap_matrix.columns[bar_col_start_idx::]

    return overlap_matrix.select(
        bar_cols
    ).sum()

create_bar_sizes(overlap_matrix)

index,time_start,time_end,bar_0_overlap,bar_1_overlap,bar_2_overlap
u32,str,str,i32,i32,i32
28,null,null,2,5,6


In [4]:
def create_concurrent_overlap(
    overlap_matrix,
    bar_col_start_idx: int = 0,
    drop_bar_cols: bool = True,
    concurrent_alias: str = "concurrency_count",
):
    """
    Compute concurrent overlap across all bars for each time interval.
    """
    bar_cols = overlap_matrix.columns[bar_col_start_idx::]

    concurrent_vector = overlap_matrix.with_columns(
        pl.sum_horizontal(bar_cols).alias(concurrent_alias)
    )

    if drop_bar_cols:
        concurrent_vector = concurrent_vector.drop(bar_cols)

    return concurrent_vector

In [94]:
from typing import Optional

def _update_chosen_indices(
    index_indicator_arr: np.array,
    index: int
) -> np.array:
    """
    Update index array
    """
    index_indicator_arr[index] = 1
    
    return index_indicator_arr

def _update_sample_array(
    sample_array,
    idx: int,
    result: int
) -> np.array:
    """
    Update sample array
    """
    sample_array[idx] = int(result)
    return sample_array
    
    
def _compute_uniqueness(
    overlap_matrix: pl.DataFrame,
    index_arr: np.array
) -> pl.DataFrame:
    """
    Compute Uniqueness
    """
    weights = 1 + np.dot(overlap_matrix, index_arr)
    
    return overlap_matrix.select(pl.all()/weights)

def _compute_average_uniqueness(
    uniqueness_matrix: pl.DataFrame,
    overlap_matrix_sizes: pl.DataFrame
) -> pl.DataFrame:
    """
    Compute Average Uniqueness
    """
    return create_bar_sizes(uniqueness_matrix)/overlap_matrix_sizes

def _compute_probabilities(
    avg_uniqueness_vector: pl.DataFrame
) -> np.array:
    """
    Compute Probabilities
    """
    return (avg_uniqueness_vector/avg_uniqueness_vector.sum_horizontal()).row(0)

def random_discrete_draw(
    elements: np.array,
    prob_dist: Optional[tuple] = None
) -> int:
    """
    Random Draw from elements using provided probability distribution
    """
    return np.random.choice(elements, p=prob_dist)

def create_sample_indices(
    overlap_matrix,
    overlap_matrix_vertical_sum,
    index_pool: np.array,
    sample_size: Optional[int] = None
) -> np.array:
    """
    Create Sample Indices
    """
    num_indices = overlap_matrix.shape[1]
    
    if sample_size is None:
        sample_size = num_indices
    
    # Initialization
    sample_indices = np.zeros(sample_size)
    chosen_indices = np.zeros(num_indices)
    
    # Initial Draw
    result = random_discrete_draw(index_pool)
    
    # Update Index Indicator Array and Sample Indices
    sample_indices = _update_sample_array(sample_indices, 0, result)
    chosen_indices = _update_chosen_indices(chosen_indices, result)
    print(f"Sample Indices: {sample_indices}")
    print(f"Chosen Indices: {chosen_indices}")
    
    num_samples = 1
    while num_samples < sample_size:
        uniqueness_matrix = _compute_uniqueness(overlap_matrix, chosen_indices)

        avg_uniqueness_vector = _compute_average_uniqueness(uniqueness_matrix, overlap_matrix_sizes)

        probs = _compute_probabilities(avg_uniqueness_vector)
        
        random_draw = random_discrete_draw(index_pool, probs)
        print(type(random_draw))
        
        # Update Index Indicator Array
        chosen_indices = _update_chosen_indices(chosen_indices, random_draw)
        sample_indices = _update_sample_array(sample_indices, num_samples, random_draw)
        
        # Increase Num Samples
        num_samples = num_samples + 1
        
        print(f"Sample Indices: {sample_indices}")
        print(f"Chosen Indices: {chosen_indices}")
        
    return sample_indices.astype(int)

In [95]:
overlap_matrix_bars = overlap_matrix.select("bar_0_overlap", "bar_1_overlap", "bar_2_overlap")

overlap_matrix_sizes = create_bar_sizes(overlap_matrix_bars)

elements = np.arange(0,3)

sample = create_sample_indices(overlap_matrix_bars, overlap_matrix_sizes, elements)
sample

Sample Indices: [1. 0. 0.]
Chosen Indices: [0. 1. 0.]
<class 'numpy.int32'>
Sample Indices: [1. 2. 0.]
Chosen Indices: [0. 1. 1.]
<class 'numpy.int32'>
Sample Indices: [1. 2. 0.]
Chosen Indices: [1. 1. 1.]


array([1, 2, 0])

numpy.ndarray

In [91]:
z[0] = 1
z

array([1, 1, 2])

In [50]:
result = random_discrete_draw(elements, probs, size = 3)
result

0

In [17]:
psi = [0, 1, 1]
import numpy as np

overlap_matrix_bars = overlap_matrix.select("bar_0_overlap", "bar_1_overlap", "bar_2_overlap")
print(overlap_matrix_bars)
df = 1 + np.dot(overlap_matrix_bars, psi)
df
print(df)
uniqueness = overlap_matrix_bars.select(pl.all()/df)
uniqueness
print(uniqueness)
overlap_matrix_sizes = create_bar_sizes(overlap_matrix_bars)
avg_uniqueness_numer = create_bar_sizes(uniqueness)
print(avg_uniqueness_numer)
print(avg_uniqueness_denom)
avg_uniqueness = avg_uniqueness_numer/overlap_matrix_sizes
avg_uniqueness

ColumnNotFoundError: bar_0_overlap

In [39]:
norm = avg_uniqueness/avg_uniqueness.sum_horizontal()
norm

bar_1_overlap,bar_2_overlap,bar_3_overlap
f64,f64,f64
0.498155,0.243542,0.258303


In [51]:
result = np.random.choice(np.arange(1,4), p=norm.row(0))
result

3

In [55]:
def draw(values, dist):
    
    return np.random.choice(values, p=dist)

iterations = 100000
counts = {"1": 0, "2": 0, "3": 0}

values = np.arange(1,4)
dist = norm.row(0)

for i in range(iterations):
    result = random_discrete_draw(elements, probs, size = 3)
    counts[str(result)] += 1

for key in counts.keys():
    counts[key] = counts[key]/iterations
    
counts

{'1': 0.49643, '2': 0.24246, '3': 0.26111}

In [19]:
all_bar_cols = overlap_matrix.columns[3::]
print(all_bar_cols)

df = overlap_matrix.with_columns(pl.sum_horizontal(all_bar_cols).alias("concurrent_sum")).drop(all_bar_cols)
df

['bar_1_overlap', 'bar_2_overlap', 'bar_3_overlap']


index,time_start,time_end,concurrent_sum
u32,str,str,i32
0,"""10:00""","""10:02""",1
1,"""10:02""","""10:03""",2
2,"""10:03""","""10:04""",2
3,"""10:04""","""10:05""",2
4,"""10:05""","""10:08""",2
5,"""10:08""","""10:10""",2
6,"""10:10""","""10:12""",1
7,"""10:12""","""10:15""",1


In [14]:
from tsuro.utils import transform_columns_to_index
index = pl.Series(values = ["9:55", "10:00", "10:02", "10:05", "10:08", "10:10", "10:12", "10:05"])

df = pl.DataFrame({
        "volume": [10, 20, 50],
        "time_start": ["10:00", "10:05", "10:10"],
        "time_end": ["10:10", "10:10", "10:15"]
        }
)

df2, index = transform_columns_to_index(df, columns = ["time_start", "time_end"], index = index, return_index = True)
df2

TypeError: transform_columns_to_index() got an unexpected keyword argument 'columns'

In [11]:
df1, index = transform_columns_to_index(df, columns = ["time_start", "time_end"], return_index = True)
df1

volume,time_start,time_end,time_start_index,time_end_index
i64,str,str,u32,u32
10,"""10:00""","""10:10""",0,2
20,"""10:05""","""10:10""",1,2
50,"""10:10""","""10:15""",2,3


In [12]:
index

values,index
str,u32
"""10:00""",0
"""10:05""",1
"""10:10""",2
"""10:15""",3


In [19]:
index = pl.Series(values = ["10:15", "10:02", "10:05", "10:08", "10:10", "10:12", "10:00"])
index = index.sort()
print(index)

index = index.alias("values")
print(index)
index = pl.DataFrame(index)
index = index.with_columns(pl.int_range(pl.len(), dtype=pl.UInt32).alias("index"))
index

shape: (7,)
Series: '' [str]
[
	"10:00"
	"10:02"
	"10:05"
	"10:08"
	"10:10"
	"10:12"
	"10:15"
]
shape: (7,)
Series: 'values' [str]
[
	"10:00"
	"10:02"
	"10:05"
	"10:08"
	"10:10"
	"10:12"
	"10:15"
]


values,index
str,u32
"""10:00""",0
"""10:02""",1
"""10:05""",2
"""10:08""",3
"""10:10""",4
"""10:12""",5
"""10:15""",6


In [3]:
trades_df = load_cba_trades()
trades_df

,time,price,volume,dollar_value,market,condition
i64,str,f64,i64,f64,str,str
0,"""6/10/2021 17:00""",100.08,12904,1.2914e6,"""ASX""","""ET XT"""
1,"""6/10/2021 16:54""",100.08,19280,1929542.4,"""ASX""","""ET XT"""
2,"""6/10/2021 16:47""",100.3127,18517,1.8575e6,"""CXA""","""SX XT"""
3,"""6/10/2021 16:46""",100.08,5364,536829.12,"""CXA""",null
4,"""6/10/2021 16:45""",100.08,24624,2.4644e6,"""ASX""","""ET XT"""
…,…,…,…,…,…,…
66572,"""6/10/2021 7:05""",95.01,6000,570060.0,"""ASX""","""EC XT"""
66573,"""6/10/2021 7:05""",94.01,1200,112812.0,"""ASX""","""EC"""
66574,"""6/10/2021 7:05""",94.01,18700,1.757987e6,"""ASX""","""EC XT"""


In [4]:
# Convert "Time" Column into Datetime Object
trades_df = cast_strings_to_datetime(trades_df, columns = ["time"])
trades_df = trades_df.rename(
   {"time": "date_time"}
)


# Grab Time Fields
trades_df = trades_df.with_columns(
    pl.col("date_time").dt.time().alias("time"),
    pl.col("date_time").dt.hour().alias("hour"),
    pl.col("date_time").dt.minute().alias("minute"),
    pl.col("date_time").dt.second().alias("second"),
    pl.col("date_time").dt.millisecond().alias("millisecond")
)

# Market Hours
trades_df = trades_df.filter(
    pl.col("time").is_between(
        time(9,0), time(17,0)
    )
)
trades_df

,date_time,price,volume,dollar_value,market,condition,time,hour,minute,second,millisecond
i64,datetime[μs],f64,i64,f64,str,str,time,i8,i8,i8,i32
0,2021-06-10 17:00:00,100.08,12904,1.2914e6,"""ASX""","""ET XT""",17:00:00,17,0,0,0
1,2021-06-10 16:54:00,100.08,19280,1929542.4,"""ASX""","""ET XT""",16:54:00,16,54,0,0
2,2021-06-10 16:47:00,100.3127,18517,1.8575e6,"""CXA""","""SX XT""",16:47:00,16,47,0,0
3,2021-06-10 16:46:00,100.08,5364,536829.12,"""CXA""",null,16:46:00,16,46,0,0
4,2021-06-10 16:45:00,100.08,24624,2.4644e6,"""ASX""","""ET XT""",16:45:00,16,45,0,0
…,…,…,…,…,…,…,…,…,…,…,…
66537,2021-06-10 10:02:00,102.0,4,408.0,"""ASX""",null,10:02:00,10,2,0,0
66538,2021-06-10 10:02:00,102.0,30,3060.0,"""ASX""",null,10:02:00,10,2,0,0
66539,2021-06-10 10:02:00,102.0,215,21930.0,"""ASX""",null,10:02:00,10,2,0,0


In [5]:
from tsuro.data_structures import StandardBars

bars = StandardBars(datetime_col = "date_time", price_col = "price", volume_col = "volume")

In [6]:
pdf = bars.create_volume_bars(
    trades_df,
    order_by = "date_time"
)
pdf

bar_index,open_price,high_price,low_price,close_price,datetime_start,datetime_end
i32,f64,f64,f64,f64,datetime[μs],datetime[μs]
3,102.5,102.59,102.01,102.07,2021-06-10 10:05:00,2021-06-10 10:10:00
12,101.02,101.06,100.76,100.815,2021-06-10 10:44:00,2021-06-10 10:47:00
9,100.85,101.08,100.72,100.98,2021-06-10 10:28:00,2021-06-10 10:32:00
30,99.93,99.95,99.64,99.65,2021-06-10 13:24:00,2021-06-10 13:36:00
21,100.2,100.275,99.95,100.16,2021-06-10 11:29:00,2021-06-10 11:41:00
…,…,…,…,…,…,…
50,100.08,100.08,100.08,100.08,2021-06-10 16:10:00,2021-06-10 16:10:00
56,99.9681,99.9681,99.9681,99.9681,2021-06-10 16:14:00,2021-06-10 16:14:00
38,99.98,100.01,99.84,100.01,2021-06-10 15:02:00,2021-06-10 15:14:00


In [9]:
from tsuro.sampling import create_overlap_matrix

import time

time0 = time.time()
overlap_matrix, bars_df = create_overlap_matrix(
    pdf,
    time_start_col = "datetime_start",
    time_end_col = "datetime_end",
    index_col = "bar_index",
    return_bars = True
)
time1 = time.time()
print(f"Time: {time1-time0}s")
overlap_matrix

Time: 0.04300689697265625s


index,time_start,time_end,bar_3_overlap,bar_12_overlap,bar_9_overlap,bar_30_overlap,bar_21_overlap,bar_6_overlap,bar_18_overlap,bar_27_overlap,bar_15_overlap,bar_0_overlap,bar_24_overlap,bar_39_overlap,bar_42_overlap,bar_36_overlap,bar_60_overlap,bar_45_overlap,bar_63_overlap,bar_51_overlap,bar_48_overlap,bar_33_overlap,bar_54_overlap,bar_13_overlap,bar_22_overlap,bar_28_overlap,bar_25_overlap,bar_16_overlap,bar_19_overlap,bar_1_overlap,bar_7_overlap,bar_31_overlap,bar_4_overlap,bar_10_overlap,bar_34_overlap,bar_64_overlap,bar_52_overlap,bar_40_overlap,bar_37_overlap,bar_46_overlap,bar_49_overlap,bar_58_overlap,bar_61_overlap,bar_43_overlap,bar_55_overlap,bar_20_overlap,bar_2_overlap,bar_26_overlap,bar_8_overlap,bar_23_overlap,bar_11_overlap,bar_17_overlap,bar_14_overlap,bar_32_overlap,bar_29_overlap,bar_5_overlap,bar_47_overlap,bar_62_overlap,bar_41_overlap,bar_44_overlap,bar_50_overlap,bar_56_overlap,bar_38_overlap,bar_53_overlap,bar_35_overlap
u32,datetime[μs],datetime[μs],i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32
0,2021-06-10 10:02:00,2021-06-10 10:05:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2021-06-10 10:05:00,2021-06-10 10:10:00,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2021-06-10 10:10:00,2021-06-10 10:14:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2021-06-10 10:14:00,2021-06-10 10:17:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,2021-06-10 10:17:00,2021-06-10 10:21:00,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
43,2021-06-10 15:59:00,2021-06-10 16:10:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
46,2021-06-10 16:15:00,2021-06-10 16:17:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
50,2021-06-10 16:22:00,2021-06-10 16:26:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [8]:
bars_df

bar_index,open_price,high_price,low_price,close_price,datetime_start,datetime_end,datetime_start_index,datetime_end_index
i32,f64,f64,f64,f64,datetime[μs],datetime[μs],u32,u32
27,99.92,99.98,99.77,99.91,2021-06-10 12:44:00,2021-06-10 12:55:00,25,26
18,99.9,100.01,99.65,99.68,2021-06-10 11:09:00,2021-06-10 11:13:00,16,17
9,100.85,101.08,100.72,100.98,2021-06-10 10:28:00,2021-06-10 10:32:00,7,8
30,99.93,99.95,99.64,99.65,2021-06-10 13:24:00,2021-06-10 13:36:00,28,29
3,102.5,102.59,102.01,102.07,2021-06-10 10:05:00,2021-06-10 10:10:00,1,2
…,…,…,…,…,…,…,…,…
38,99.98,100.01,99.84,100.01,2021-06-10 15:02:00,2021-06-10 15:14:00,36,37
53,100.08,100.08,100.08,100.08,2021-06-10 16:10:00,2021-06-10 16:10:00,44,44
35,99.97,100.0,99.73,99.77,2021-06-10 14:24:00,2021-06-10 14:36:00,33,34


In [9]:
bars_start_times = pdf.select("datetime_start").unique()
bars_end_times = pdf.select("datetime_end").unique()
bars_end_times

datetime_end
datetime[μs]
2021-06-10 16:10:00
2021-06-10 16:20:00
2021-06-10 10:08:00
2021-06-10 15:38:00
2021-06-10 16:43:00
…
2021-06-10 10:47:00
2021-06-10 13:43:00
2021-06-10 14:58:00
